# Language Modeling using Ngram

In this Exercise, you are going to use NLTK which is a natural language processing library for python to create a bigram language model and its variation. You will build one model for each of the following type and calculate their perplexity:
- Unigram Model
- Bigram Model
- Bigram Model with Laplace smoothing
- Bigram Model with Interpolation
- Bigram Model with Kneser-ney Interpolation
- Neural LM (optional)



In [1]:
#download corpus
!wget --no-check-certificate https://github.com/ekapolc/nlp_2019/raw/master/HW4/BEST2010.zip
!unzip BEST2010.zip

--2021-02-14 12:55:22--  https://github.com/ekapolc/nlp_2019/raw/master/HW4/BEST2010.zip
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ekapolc/nlp_2019/master/HW4/BEST2010.zip [following]
--2021-02-14 12:55:22--  https://raw.githubusercontent.com/ekapolc/nlp_2019/master/HW4/BEST2010.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7423530 (7.1M) [application/zip]
Saving to: ‘BEST2010.zip’

BEST2010.zip        100%[===================>]   7.08M  --.-KB/s    in 0.1s    

2021-02-14 12:55:23 (53.4 MB/s) - ‘BEST2010.zip’ saved [7423530/7423530]

Archive:  BEST2010.zip
   creating: BEST2010/
  i

In [2]:
#First we import necessary library such as math, nltk, bigram, and collections.
import math
import nltk
import io
import random
from random import shuffle
from nltk import bigrams, trigrams
from collections import Counter, defaultdict
random.seed(999)

BEST2010 is a free Thai NLP dataset by NECTEC usually use as a standard benchmark for various NLP tasks includeing language modeling. BEST2010 is separated into 4 domain article, encyclopedia, news and novel. The data is already  tokenized using '|' as a separator.

For example,

ตาม|ที่|นางประนอม ทองจันทร์| |กับ| |ด.ช.กิตติพงษ์ แหลมผักแว่น| |และ| |ด.ญ.กาญจนา กรองแก้ว| |ป่วย|สงสัย|ติด|เชื้อ|ไข้|ขณะ|นี้|ยัง|ไม่|ดี|ขึ้น|

In [3]:
# We choose news domain as our dataset
best2010=[]
fp= io.open('BEST2010/news.txt','r',encoding='utf-8')
for i,line in enumerate(fp):
    best2010.append(line.strip()[:-1])
fp.close()
all_vocabulary =set()
total_word_count =0
for line in best2010:
    for word in line.split('|'):        
        all_vocabulary.add(word)
        total_word_count+=1

In [4]:
#For simplicity, we assumes that each line is a sentence.
print ('Total sentences in BEST2010 news dataset :\t'+ str(len(best2010)))
print ('Total word counts in BEST2010 news dataset :\t'+ str(total_word_count))
print ('Total vocabulary in BEST2010 news dataset :\t'+ str(len(all_vocabulary)))

Total sentences in BEST2010 news dataset :	30969
Total word counts in BEST2010 news dataset :	1660190
Total vocabulary in BEST2010 news dataset :	35488


We separate out input into 2 sets, train and test data with 70:30 ratio

In [5]:
sentences = best2010
# The data is separated to train and test set with 70:30 ratio.
train = sentences[:int(len(sentences)*0.7)]
test = sentences[int(len(sentences)*0.7):]

#Training data
train_vocabulary =set()
train_word_count =0
for line in train:
    for word in line.split('|'):        
        train_vocabulary.add(word)
        train_word_count+=1
print ('Total sentences in BEST2010 news training dataset :\t'+ str(len(train)))
print ('Total word counts in BEST2010 news training dataset :\t'+ str(train_word_count))
print ('Total vocabuary in BEST2010 news training dataset :\t'+ str(len(train_vocabulary)))
# We will use 1/vocab_size as a default value for unknown word
unk_value = math.pow(len(train_vocabulary),-1)

Total sentences in BEST2010 news training dataset :	21678
Total word counts in BEST2010 news training dataset :	1042797
Total vocabuary in BEST2010 news training dataset :	26240


# Unigram

In this section, we will demonstrate how to build a unigram language model <br>
**Important note:** <br>
**\<s\>** = sentence start symbol <br>
**\</s\>** = sentence end symbol 

In [6]:
def getUnigramModel(data):
    model = defaultdict(lambda: 0)
    word_count =0
    for sentence in data:
        sentence +=  u'|</s>' #for unigram model we can always ignore <s>, since p(w0=<s>)=1
        for w1 in sentence.split('|'):
            model[w1] +=1.0
            word_count+=1
    for w1 in model:
        model[w1] = model[w1]/(word_count)
    return model

In [7]:
model = getUnigramModel(train)

In [8]:
def getLnValue(x):
    if x >0.0:
        return math.log(x)
    else:
        return math.log(unk_value)

In [9]:
#problability of 'นายก'
print(getLnValue(model[u'นายก']))
#for example, problability of 'นายกรัฐมนตรี' which is an unknown word is equal to
print(getLnValue(model[u'นายกรัฐมนตรี']))
#problability of 'นายก' 'ได้' 'ให้' 'สัมภาษณ์' 'กับ' 'สื่อ'
prob = getLnValue(model[u'นายก'])+getLnValue(model[u'ได้'])+ getLnValue(model[u'ให้'])+getLnValue(model[u'สัมภาษณ์'])+getLnValue(model[u'กับ'])+getLnValue(model[u'สื่อ'])+getLnValue(model['</s>'])
print ('Problability of a sentence', math.exp(prob))

-6.551526663995246
-10.175040243058024
Problability of a sentence 5.617210748667918e-18


# Perplexity

In order to compare language model we need to calculate perplexity. In this task you should write a perplexity calculation code for the unigram model. The result perplexity should be around 556.39 and
476.07 on train and test data.

# TODO #0
**Fill your name and ID here** <br>
**Name**: Ravipas Aphikulvanich
<br>
**ID**: 6031050121

## TODO #1 **Calculate perplexity**

In [10]:
import math
def calculate_sentence_ln_prob(sentence, model):
  
  out = 0
  count = 0
  words = sentence.split('|')
  for k in words:

    out += getLnValue(model[k])
    
  return out,len(words)

    


def perplexity(test,model):
    out = 0
    count = 0

    for i,sentence in enumerate(test):
      sentence += '|</s>'
     
     
      n = len(sentence.split('|'))

      out+=   calculate_sentence_ln_prob(sentence,model)[0] 
      count += calculate_sentence_ln_prob(sentence,model)[1] 
    
    return math.e ** ( -out/count)
  

In [11]:
print(perplexity(train,model))
print(perplexity(test,model))

556.3925994212193
476.06878923035305


## TODO #2 **Please explain why this model give us such a high perplexity.**

**Your answer**: Because the model are considered by unigram probability which not include information of consecutive words so every calculation that happened, only use independent probability of individual word combine with each other in sentence. 
They are no clue which next word has more or less probability so it make this model to has high perplexity because we have to take many time for random to guess a correct and valid sentence but if we use priciple of bigram the model may be has a less perplexity  

# Bigram

Next, you will create a better language model than a unigram (which is not much to compare with). But first, it is very tedious to count every pair of words that occur in our corpus by ourselves. In this case, nltk provide us a simple library which will do it for us.

In [12]:
#example of nltk usage for bigram
sentence = 'I always search google for an answer .'

print('This is how nltk generate bigram.')
for w1,w2 in bigrams(sentence.split(), pad_right=True, pad_left=True):
    print (w1,w2)
print('None is used as a start and end of sentence symbol.')

This is how nltk generate bigram.
None I
I always
always search
search google
google for
for an
an answer
answer .
. None
None is used as a start and end of sentence symbol.


Now, you should be able to implement a bigram model by yourself. Also, you must create a new perplexity calculation for bigram. The result perplexity should be around 58.78 and 146.26 on train and test data.

In [ ]:
len(train)

21678

In [ ]:
model

In [ ]:
model = defaultdict(lambda: 0)

In [ ]:
 model = defaultdict(lambda: defaultdict(lambda: 0))
    

## TODO #3 **Create a Bigram Model**

In [13]:
model['5']= {'5':'3'}

In [14]:
def getBigramModel(data):
  count = defaultdict(lambda: 0)
  model = defaultdict(lambda: defaultdict(lambda: 0))
 
  for sentence in data:
    # sentence += '|<\s>'
    # sentence += '<s>|'
    for w1,w2 in bigrams(sentence.split('|'), pad_right=True, pad_left=True):
      model[w1][w2] += 1
      count[w1] +=  1

  for key in model:
    for key2 in model[key]:
      model[key][key2] /= count[key]


  return model

model = getBigramModel(train)

In [ ]:
model

'หลัง|เข้า|เยี่ยม|ดู|อาการ|ผู้|ป่วย|แล้ว| |น.พ.จรัล|ประชุม|ร่วม|กับ|เจ้าหน้าที่|ทุก|ฝ่าย| |เพื่อ|สรุป|ผล|การ|ดำเนิน|การ| |รวม|ทั้ง|สอบสวน|โรค|ก่อน|ที่|ผู้|ป่วย|จะ|ถูก|ส่ง|มา|รักษา|ตัว| |จาก|นั้น|ร่วม|กัน|แถลง|ข่าว| |โดย| |น.พ.จรัล|กล่าว|ว่า| |ขณะ|นี้|ผู้|ป่วย|ทั้ง| |3| |ราย| |อาการ|ยัง|ทรง| |โดย|ใน|ราย|ของ| |ด.ช.กิตติพงษ์| |กับ| |ด.ญ.กาญจนา| |ปอด|หาย|เป็น|ปกติ|แล้ว| |คาด|ว่า|จะ|กลับ|บ้าน|ได้|ใน|ไม่|ช้า|นี้| |แต่|ใน|ราย|ของ|นางประนอม|อาการ|ยัง|น่า|เป็นห่วง| |ซึ่ง|ทั้ง| |3| |ราย| |ใน|ชั้น|นี้|ถือ|ว่า|เป็น|ผู้|ป่วย|อยู่|ใน|ขั้น|น่า|สงสัย|อาจ|ติด|เชื้อ|ไข้|หวัด|นก| |เพราะ|ตรวจ|พบ|ผู้|ป่วย|มี|อาการ|ปอด|บวม|ปอด|อักเสบ| |เนื่อง|จาก|ติด|เชื้อ|ไวรัส| |แต่|ยัง|สรุป|ไม่|ได้|ว่า|ติด|เชื้อ|ไข้|หวัด|นก|แน่ชัด|หรือ|ไม่| |ต้อง|รอ|ผล|ตรวจ|จาก|ห้อง|ปฏิบัติการ|ที่|ได้|ส่ง|ตัวอย่าง|เลือด| |ไป|ตรวจ|พิสูจน์|ที่|กรมวิทยาศาสตร์การแพทย์กระทรวงสาธารณสุข| |และ| |รพ.ศิริราช| |ซึ่ง|จะ|ทราบ|ผล|ใน|วัน|สอง|วัน|นี้'

In [ ]:
for w1,w2 in bigrams(sentence.split('|'), pad_right=True, pad_left=True):
      model[w1][w2] += 1
      count +=  1

## TODO #4 **Calculate Perplexity for Bigram Model**



In [17]:
def calculate_sentence_ln_prob(sentence, model):
    out = 0

    for w1,w2 in bigrams(sentence.split('|'), pad_right=True, pad_left=True):
      # model[w1][w2] += 1

      out += getLnValue(model[w1][w2])
    return out

def perplexity(test,model):
    count = 0
    out = 0
    

    for i,sentence in enumerate(test):
      # sentence += '|</s>'
      # sentence = '<s>|' + sentence 
     
      n = len(sentence.split('|')) + 1
      count += n

      out+=   calculate_sentence_ln_prob(sentence,model)

    
    return math.e ** ( -out/count)

In [18]:
print (perplexity(train,model) )
print (perplexity(test, model))

# 58.78942889767147
# 146.26539331038614

58.789428897671456
146.26539331038612


# Smoothing

In [19]:
def getBigramModelLaplace(data):
  count = defaultdict(lambda: 1)
  model = defaultdict(lambda: defaultdict(lambda: 1))
 
  for sentence in data:
    # sentence += '|<\s>'
    # sentence += '<s>|'
    for w1,w2 in bigrams(sentence.split('|'), pad_right=True, pad_left=True):
      model[w1][w2] += 1
      count[w1] +=  1

  for key in model:
    for key2 in model[key]:
      model[key][key2] /= count[key]


  return model

model = getBigramModel(train)

## TODO #5 **Bigram with Laplace smoothing (Add-One Smoothing)**

In [20]:
#Laplace Smoothing
def getBigramWithLaplaceSmoothing(data):
  count = defaultdict(lambda: 0)
  model = defaultdict(lambda: defaultdict(lambda: 0))
  unk_value = math.pow(len(train_vocabulary),-1)
  for sentence in data:
    # sentence += '|<\s>'
    # sentence += '<s>|'
    for w1,w2 in bigrams(sentence.split('|'), pad_right=True, pad_left=True):
      model[w1][w2] += 1
      count[w1] +=  1

  for key in model:
    for key2 in model[key]:
      model[key][key2] = (model[key][key2] + 1)/(count[key] + len(count))
    



  return model

model = getBigramWithLaplaceSmoothing(train)
print (perplexity(train,model) )
print (perplexity(test, model))

# 974.8134581679766
# 1098.1622194979489

974.8134581679763
1098.1622194979484


In [21]:
def getUnigramModel2(data,word_count):


    model = defaultdict(lambda: 0)

    for sentence in data:
        # sentence +=  u'|</s>' #for unigram model we can always ignore <s>, since p(w0=<s>)=1
        for w1,w2 in bigrams(sentence.split('|'), pad_right=True, pad_left=True):
            model[w1] +=1.0
    for w1 in model:
        model[w1] = model[w1]/word_count
    return model

In [ ]:
unk_value

3.8109756097560976e-05

## TODO #6 **Bigram with Interpolation**
lambda value is 0.7 for bigram, 0.25 for unigram, and 0.05 for unknown word

In [22]:
#interpolation
def getBigramWithInterpolation(data):
  # count = defaultdict(lambda: 0)
  model = getBigramModel(data)
  unigram = getUnigramModel2(data,train_word_count)
  

  for key in model:
    for key2 in model[key]:
      model[key][key2] *= 0.70

  for key in model:
    for key2 in model[key]:
      model[key][key2] += 0.25*unigram[key2]

  for key in model:
    for key2 in model[key]:
      model[key][key2] += 0.05*unk_value

    
  return model
  

    
model = getBigramWithInterpolation(train)
print (perplexity(train,model))        
print (perplexity(test,model))

# 73.38409869825665
# 172.67485908813356

73.38409869825664
172.6748590881335


the result perplexity on training and testing should be 

    974.81, 1098.16 for Laplace smoothing
    73.38, 172.67 for Interpolation

# Language modeling on multiple domains

Sometimes, we do not have enough data to create a language model for a new domain. In that case, we can improvised by combining several models to improve result on the new domain.

In this exercise you will try to merge two language models from news and article domains to create a language model for the encyclopedia domain.

In [23]:
# create article data
encyclo_data=[]
fp= io.open('BEST2010/encyclopedia.txt','r',encoding='utf-8')
for i,line in enumerate(fp):
    encyclo_data.append(line.strip()[:-1])
fp.close()

First, you should try to calculate perplexity of your bigram with interpolation using "news data" (train) on "encyclopedia data" (test). The result perplexity should be around 727.35.

For your information, a bigram model with interpolation using "ariticle data" (train) to test on "encyclopedia data" (test) has a perplexity of 505.79.

In [24]:
# print perplexity of bigram with interpolation on article data        
# 727.3502637212223
print (perplexity(encyclo_data,model))

727.3502637212221


In [25]:
# create article data
data=[]

fp= io.open('BEST2010/article.txt','r',encoding='utf-8')
for i,line in enumerate(fp):
    data.append(line.strip()[:-1])
fp.close()


fp= io.open('BEST2010/news.txt','r',encoding='utf-8')
for i,line in enumerate(fp):
    data.append(line.strip()[:-1])
fp.close()


all_vocabulary =set()
total_word_count =0
for line in data:
    for word in line.split('|'):        
        all_vocabulary.add(word)
        total_word_count+=1

In [ ]:
len(all_vocabulary)

52581

In [26]:
##interpolation
def getBigramWithInterpolation2(data,n_vocab):
  # count = defaultdict(lambda: 0)
  unk_value = math.pow(n_vocab,-1)
  model = getBigramModel(data)
  unigram = getUnigramModel2(data,train_word_count)
  

  for key in model:
    for key2 in model[key]:
      model[key][key2] *= 0.70

  for key in model:
    for key2 in model[key]:
      model[key][key2] += 0.25*unigram[key2]

  for key in model:
    for key2 in model[key]:
      model[key][key2] += 0.05*unk_value

    
  return model
  

    

# 974.8134581679766
# 1098.1622194979489

## TODO #7 
Write a model that produce 450.0 or less perplexity on encyclopedia data without using data from the encyclopedia as training data. (Hint : Try to combine a model with news data and a model with article data together.)

In [27]:
combined_model = getBigramWithInterpolation2(data,len(all_vocabulary))
# 428.85251789073953 (on combined data)
print('Perplexity of combine Bigram model with interpolation smoothing on encyclopedia test data',perplexity(encyclo_data, combined_model))

Perplexity of combine Bigram model with interpolation smoothing on encyclopedia test data 364.0662576538735


## TODO #8 
## Kneser-ney on "News"

<!-- Reimplement equation 4.33 in SLP textbook (https://lagunita.stanford.edu/c4x/Engineering/CS-224N/asset/slp4.pdf) -->

Implement Bigram Knerser-ney LM. The result perplexity should be around 55.06 and 140.76 on train and test data. 


In [28]:
def getBigramCount(data):


  fw = defaultdict(lambda: defaultdict(lambda: 0))
  bw = defaultdict(lambda: defaultdict(lambda: 0))
 
  for sentence in data:

    for w1,w2 in bigrams(sentence.split('|'), pad_right=True, pad_left=True):
      fw[w1][w2]
      bw[w2][w1] += 1
  return fw,bw 

def getUniqueAndCount(dic):
  unique = defaultdict(lambda: 0)
  count = defaultdict(lambda: 0)
  for k1 in dic:
    unique[k1]= len(dic[k1])
    for k2 in dic[k1]:
      count[k1] += dic[k1][k2]

  return unique, count



In [52]:
unigram = defaultdict(lambda: 0.0)
bigram= defaultdict(lambda: defaultdict(lambda: 0))


for sentence in data:
    for w1, w2 in bigrams(sentence.split('|'), pad_right=True, pad_left=True): 
        bigram[w1][w2] += 1.0
        unigram[w1] += 1.0


bigram_fw = defaultdict(lambda: 0.0)
bigram_bw = defaultdict(lambda: 0.0)

for k1 in bigram:
  for k2 in bigram[k1]:
    if bigram[k1][k2] == 0:
      break
    bigram_fw[k1] += 1
    bigram_bw[k2] += 1#
 
for k1 in bigram:
  if k1 in unigram:
    
    for k2 in bigram[k1]:

      bigram[k1][k2] = (max(bigram[k1][k2]-0.75,0)/unigram[k1]) +  (0.75 / unigram[k1]) * bigram_bw[k2] *  bigram_bw[k1]/float(len(bigram_fw))
print (perplexity(train,model))        
print (perplexity(test,model))
# 71.14054002208687
# 174.02464248000433

73.38409869825664
172.6748590881335


0.0

## TODO #9
## Neural LM (optional)
do it on news corpus that we splitted into train and test sets at the beginning of this exercise. 

In [ ]:
#there are many ways to do this. e.g.:
#https://machinelearningmastery.com/develop-word-based-neural-language-models-python-keras/